# loading from Hugging Face and inference
다음은 Hugging Face에서 LoRA adapter을 불러와서 대화 생성하는 기본 과정입니다.

GPU는 A100 GPU가 아닌 T4 GPU를 사용했습니다.

라이브러리를 설치하고 hugging face에서 adapter 불러오고 기본 모델 불러와서 model 구성하는 것까지는 inference 전 공통 과정입니다.

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# 1. Hugging Face에 올린 LoRA adapter 경로
adapter_repo = "wjdbin217/DialogueGenModel_finetuning_medical3"

# 2. LoRA 원본 base model 불러오기
base_model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(adapter_repo)
# 일반적으로 tokenizer은 base model 기준으로 가져오긴 함. 지금은 adapter repo에서 가져옴.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# 3. LoRA adapter 적용
model = PeftModel.from_pretrained(base_model, adapter_repo)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

다음은 기본적인 inference 과정입니다. 참고만 해주세요.

In [8]:
# 4. Inference
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        #         "사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다."
        # "사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다.."
        # "예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. "
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},
    {"role": "user", "content": "경희내과 맞나요? 저 오늘 예약한 홍길동인데, 시간을 좀 바꿔야 할 것 같아요."},
    {"role": "assistant", "content": "네, 고객님. 경희내과입니다. 본인 확인되셨고요. 어떤 시간대로 변경을 희망하시나요?"},
    {"role": "user", "content": "네. 오후로요. 딱 정해진 시간은 없는데... 혹시 추천해주실 만한 시간 있으세요?"},
]

# Chat 템플릿 적용
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )


end_time = time.time()
inference_time = end_time - start_time

# 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=True)        # 입력 프롬프트 (prompt) + 생성된 응답 전체를 포함한 토큰 시퀀스 전체를 디코딩한 문자열임.
print("- - - - - - - - - - - - - - - - - 전체 응답 - - - - - - - - - - - - - - - - - ")
print(result)

print(f"\n 추론 시간: {inference_time :.2f}초")



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


- - - - - - - - - - - - - - - - - 전체 응답 - - - - - - - - - - - - - - - - - 
system

당신은 경희내과의 병원 콜센터 상담원입니다.모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다.user

경희내과 맞나요? 저 오늘 예약한 홍길동인데, 시간을 좀 바꿔야 할 것 같아요.assistant

네, 고객님. 경희내과입니다. 본인 확인되셨고요. 어떤 시간대로 변경을 희망하시나요?user

네. 오후로요. 딱 정해진 시간은 없는데... 혹시 추천해주실 만한 시간 있으세요?assistant

네, 성함과 예약 시간 알려주시면 가장 편한 시간을 안내해드릴게요.

 추론 시간: 61.80초


# 실제로 처음부터 끝까지 모델만 사용해서 대화 이어나가는 과정
prompt 형식은 GitHub를 참고해주세요.

- user 발화는 training data를 참고해서 직접 구성하였고, assistant 발화는 model만을 사용해서 생성해냈습니다.
- input prompt (코드 내에서는 messages)를 제외한 나머지 코드는 모든 대화 턴에서 동일합니다.
- prompt의 "system"은 매 대화 턴마다 동일하게 작성되어 있습니다.
- prompt의 "user", "assistant"는 매 대화 턴마다 새로 생성한 발화까지 포함시켜서 멀티턴 대화쌍으로 구성합니다. (당연히 첫번째 대화 턴은 user 로만 구성되어 있습니다.)


### 1. 첫 번째 대화 턴

In [9]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},     # user 발화 1
]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?

 추론 시간: 36.15초


### 2. 두 번째 대화 턴

In [11]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},                           # user 발화 1
    {"role": "assistant", "content": "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"},     # 생성한 assistant 발화 1
    {"role": "user", "content": "아, 네. 예약 변경하려고 하는데요."},                           # user 발화 2
]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 성함 말씀해주시겠어요?

 추론 시간: 39.06초


### 3. 세 번째 대화 턴

In [13]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},                           # user 발화 1
    {"role": "assistant", "content": "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"},     # 생성한 assistant 발화 1
    {"role": "user", "content": "아, 네. 예약 변경하려고 하는데요."},                           # user 발화 2
    {"role": "assistant", "content": "네, 성함 말씀해주시겠어요?"},                             # 생성한 assistant 발화 2
    {"role": "user", "content": "네. 홍길동입니다. 오늘 오전 11시 예약이었어요."},              # user 발화 3

]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 확인되었습니다. 언제로 변경해드릴까요?

 추론 시간: 57.08초


### 4. 네 번째 대화 턴

In [15]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},                           # user 발화 1
    {"role": "assistant", "content": "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"},     # 생성한 assistant 발화 1
    {"role": "user", "content": "아, 네. 예약 변경하려고 하는데요."},                           # user 발화 2
    {"role": "assistant", "content": "네, 성함 말씀해주시겠어요?"},                             # 생성한 assistant 발화 2
    {"role": "user", "content": "네. 홍길동입니다. 오늘 오전 11시 예약이었어요."},              # user 발화 3
    {"role": "assistant", "content": "네, 확인되었습니다. 언제로 변경해드릴까요?"},             # 생성한 assistant 발화 3
    {"role": "user", "content": "네. 오후 3시로 변경 가능할까요?"},                             # user 발화 4


]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 가능합니다. 오후 3시로 변경해드릴게요. 다른 궁금한 점 있으신가요?

 추론 시간: 74.89초


### 5. 다섯 번째 대화 턴

In [16]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},                                               # user 발화 1
    {"role": "assistant", "content": "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"},                         # 생성한 assistant 발화 1
    {"role": "user", "content": "아, 네. 예약 변경하려고 하는데요."},                                               # user 발화 2
    {"role": "assistant", "content": "네, 성함 말씀해주시겠어요?"},                                                 # 생성한 assistant 발화 2
    {"role": "user", "content": "네. 홍길동입니다. 오늘 오전 11시 예약이었어요."},                                  # user 발화 3
    {"role": "assistant", "content": "네, 확인되었습니다. 언제로 변경해드릴까요?"},                                 # 생성한 assistant 발화 3
    {"role": "user", "content": "네. 오후 3시로 변경 가능할까요?"},                                                 # user 발화 4
    {"role": "assistant", "content": "네, 가능합니다. 오후 3시로 변경해드릴게요. 다른 궁금한 점 있으신가요?"},      # 생성한 assistant 발화 4
    {"role": "user", "content": "아, 네. 진료 끝나고 진단서 바로 받을 수 있죠?"},                                   # user 발화 5
]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 진료 끝나고 바로 진단서 발급해드릴 수 있습니다.

 추론 시간: 80.42초


### 6. 여섯 번째 대화 턴

In [17]:
messages = [      # 결과가 prompt system 내용을 너무 탐. prompt system을 너무 구체적으로 작성 X
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},

    {"role": "user", "content": "안녕하세요. 거기 경희내과 맞나요?"},                                               # user 발화 1
    {"role": "assistant", "content": "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"},                         # 생성한 assistant 발화 1
    {"role": "user", "content": "아, 네. 예약 변경하려고 하는데요."},                                               # user 발화 2
    {"role": "assistant", "content": "네, 성함 말씀해주시겠어요?"},                                                 # 생성한 assistant 발화 2
    {"role": "user", "content": "네. 홍길동입니다. 오늘 오전 11시 예약이었어요."},                                  # user 발화 3
    {"role": "assistant", "content": "네, 확인되었습니다. 언제로 변경해드릴까요?"},                                 # 생성한 assistant 발화 3
    {"role": "user", "content": "네. 오후 3시로 변경 가능할까요?"},                                                 # user 발화 4
    {"role": "assistant", "content": "네, 가능합니다. 오후 3시로 변경해드릴게요. 다른 궁금한 점 있으신가요?"},      # 생성한 assistant 발화 4
    {"role": "user", "content": "아, 네. 진료 끝나고 진단서 바로 받을 수 있죠?"},                                   # user 발화 5
    {"role": "assistant", "content": "네, 진료 끝나고 바로 진단서 발급해드릴 수 있습니다."},                        # 생성한 assistant 발화 5
    {"role": "user", "content": "아, 네. 감사합니다."},                                                             # user 발화 6
]

# Chat 템플릿 적용 및 입력 시퀀스 생성
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

import time
start_time = time.time()  # 추론시간 추적

# 입력 토큰 길이 저장해놓기
input_length = inputs["input_ids"].shape[-1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 길이가 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

end_time = time.time()
inference_time = end_time - start_time

# 생성된 토큰만 디코딩
generated_tokens = outputs[0][input_length:]    # 입력 이후만 자르기
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - ")
print(result.strip())

print(f"\n 추론 시간: {inference_time :.2f}초")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



- - - - - - - - - - - - - - - - - 생성된 발화 내용 - - - - - - - - - - - - - - - - - 
네, 감사합니다. 좋은 하루 보내세요.

 추론 시간: 94.13초


## 전체 대화 내용


user 발화 1 : "안녕하세요. 거기 경희내과 맞나요?"

assistant 발화 1 : "네, 맞습니다. 경희내과입니다. 무엇을 도와드릴까요?"

user 발화 2 : "아, 네. 예약 변경하려고 하는데요."

assistant 발화 2 : "네, 성함 말씀해주시겠어요?"

user 발화 3 : "네. 홍길동입니다. 오늘 오전 11시 예약이었어요."

assistant 발화 3 : "네, 확인되었습니다. 언제로 변경해드릴까요?"

user 발화 4 : "네. 오후 3시로 변경 가능할까요?"

assistant 발화 4 : "네, 가능합니다. 오후 3시로 변경해드릴게요. 다른 궁금한 점 있으신가요?"

user 발화 5 : "아, 네. 진료 끝나고 진단서 바로 받을 수 있죠?"

assistant 발화 5 : "네, 진료 끝나고 바로 진단서 발급해드릴 수 있습니다."

user 발화 6 : "아, 네. 감사합니다."

assistant 발화 6 : "네, 감사합니다. 좋은 하루 보내세요."

## 개선 사항
대화 턴이 계속될수록 messages가 길어져서 그만큼 추론 시간도 오래 걸림을 알 수 있었습니다.

대화 턴이 길어지면 최근 대화 턴 3쌍 정도만 사용하는 식으로도 개선할 수 있을 것 같습니다.

ex) 여섯 번째 대화 턴에서는 발화 1 ~ 6을 전부 사용하지 않고 4 ~ 6까지만 사용.
